Fiz um Conda venv com Python 3.10 e instalei o modulus normalmente.
Dependendo do que esteja na máquina as vezes é necessário instalar o setup tools, mas acredito que assim já funcione, desde rode tudo dentro do Venv, instalando os pacotes em requirements.txt 


In [1]:
#Conda env with python 3.10
#install conflict using 3.12!
%pip install -f requirements.txt
%pip install setuptools
%pip install nvidia-modulus nvidia-modulus-sym



ERROR: You must give at least one requirement to install (maybe you meant "pip install requirements.txt"?)
Note: you may need to restart the kernel to use updated packages.
  Using cached nvidia_modulus-0.7.0-py3-none-any.whl.metadata (8.6 kB)
  Using cached nvidia_modulus.sym-1.6.0-py3-none-any.whl.metadata (5.6 kB)
  Using cached torch-2.4.0-cp310-cp310-manylinux1_x86_64.whl.metadata (26 kB)
  Using cached numpy-1.24.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (5.6 kB)
  Using cached xarray-2024.7.0-py3-none-any.whl.metadata (11 kB)
  Using cached zarr-2.18.2-py3-none-any.whl.metadata (5.7 kB)
  Using cached fsspec-2024.6.1-py3-none-any.whl.metadata (11 kB)
  Using cached s3fs-2024.6.1-py3-none-any.whl.metadata (1.6 kB)
  Using cached nvidia_dali_cuda120-1.40.0-16741769-py3-none-manylinux2014_x86_64.whl
  Using cached treelib-1.7.0-py3-none-any.whl.metadata (1.3 kB)
  Using cached nvtx-0.2.10-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadat

### O Modulus soobreescreve a classe Module em torch.nn , gerando alteração mínima no código, e logo está pronto para ser parametrizado a fim de utilizar todos os benefícios do Module.

Os dois codeblocks abaixo mostrar a declaração tradicional de uma NN em pytorch e o segundo como converter esta rede para o Modulus, adicionando apenas um pouco de Metadata.

In [ ]:
#model building -> pytorch standard

import torch.nn as nn


##Pytorch
class UNet(nn.Module):
    def __init__(self, in_channels=1, out_channels=1):
        super(UNet, self).__init__()

        self.enc1 = self.conv_block(in_channels, 64)
        self.enc2 = self.conv_block(64, 128)

        self.dec1 = self.upconv_block(128, 64)
        self.dec2 = self.upconv_block(64, 32)
        self.final = nn.Conv2d(32, out_channels, kernel_size=1)

    def conv_block(self, in_channels, out_channels):
        return nn.Sequential(
            nn.Conv2d(in_channels, out_channels, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),
        )

    def upconv_block(self, in_channels, out_channels):
        return nn.Sequential(
            nn.ConvTranspose2d(in_channels, out_channels, 2, stride=2),
            nn.Conv2d(out_channels, out_channels, 3, padding=1),
            nn.ReLU(inplace=True),
        )

    def forward(self, x):
        x = self.enc1(x)
        x = self.enc2(x)
        x = self.dec1(x)
        x = self.dec2(x)
        return self.final(x)
    
    
    

In [ ]:

from dataclasses import dataclass

import torch.nn as nn

from modulus.models.meta import ModelMetaData
from modulus.models.module import Module


@dataclass
class MdlsUNetMetaData(ModelMetaData):
    name: str = "MdlsUNet"
    # Optimization
    jit: bool = False
    cuda_graphs: bool = True
    amp_cpu: bool = True
    amp_gpu: bool = True


MdlsUNet = Module.from_torch(UNet, meta=MdlsUNetMetaData)